In [ ]:
%%capture
!pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import sys
import os
import pandas as pd

In [2]:
# Config of the notebook
random_seed = 585

In [3]:
# Download latest version
stock_data = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "camnugent/sandp500",
    "all_stocks_5yr.csv",
)

# download company info
comp_info = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "paytonfisher/sp-500-companies-with-financial-information",
    "financials.csv",
)


100%|██████████| 9.60M/9.60M [00:01<00:00, 8.69MB/s]

Extracting zip of all_stocks_5yr.csv...


100%|██████████| 91.2k/91.2k [00:00<00:00, 344kB/s]


### Selected Companies

In [4]:
# Calculate the value counts for the 'Sector' column
sector_counts = comp_info['Sector'].value_counts()

In [5]:
# sample 5 top sectors and select 5 random stocks
selected_sectors = list(sector_counts.index[:3])

selected_sector_comp = comp_info[comp_info.apply(lambda row: row["Sector"] in selected_sectors, axis=1)]

# selected_companies
selected_companies = selected_sector_comp.groupby(by="Sector").sample(5, random_state=random_seed)

In [38]:
selected_tickers = list(selected_companies["Symbol"])
sampled_stock_data = stock_data[stock_data.apply(lambda row: row["Name"] in selected_tickers, axis=1)]

# rename Name to Ticker
sampled_stock_data = sampled_stock_data.rename(columns={"Name": "Ticker"})